In [27]:
library("DBI")
library("dbplyr")
library("dplyr")

con <- DBI::dbConnect(
    RPostgres::Postgres(),
    host = "localhost",
    port = 5432,
    dbname = "knock100",
    user = "guest",
    password = "guest"
)

customer_tbl <- dplyr::tbl(con, "customer")
category_tbl <- dplyr::tbl(con, "category")
product_tbl <- dplyr::tbl(con, "product")
receipt_tbl <- dplyr::tbl(con, "receipt")
store_tbl <- dplyr::tbl(con, "store")
geo_tbl <- dplyr::tbl(con, 'geocode')
DBI::dbListTables(con)

[1] "customer" "category" "geocode"  "product"  "receipt"  "store"

### R-061:
レシート明細データ（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、売上金額合計を常用対数化（底10）して顧客ID、売上金額合計とともに10件表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

In [ ]:
library(stringr)
re %>%
filter(!str_detect(customer_id, '^Z')) %>%
group_by(customer_id) %>%
summarise(amount = sum(amount, na.rm = TRUE)) %>%
mutate(log_amount = log10(amount)) %>%
head

### R-062:
レシート明細データ（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、売上金額合計を自然対数化（底e）して顧客ID、売上金額合計とともに10件表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

In [ ]:
library(stringr)
re %>%
filter(!str_detect(customer_id, '^Z')) %>%
group_by(customer_id) %>%
summarise(amount = sum(amount, na.rm = TRUE )) %>%
mutate(log_amount = log(amount)) %>%
head

### R-063:
商品データ（df_product）の単価（unit_price）と原価（unit_cost）から各商品の利益額を算出し、結果を10件表示せよ。

In [ ]:
product <- tbl(con, 'product')
product %>%
transmute(product_cd, unit_price, unit_cost, unit_profit = unit_price - unit_cost) %>%
head

### R-064:
商品データ（df_product）の単価（unit_price）と原価（unit_cost）から、各商品の利益率の全体平均を算出せよ。ただし、単価と原価には欠損が生じていることに注意せよ。

In [ ]:
product %>%
mutate(avg_profit_rate = mean(unit_price - unit_cost, na.rm = TRUE)/mean(unit_price)) %>%
summarise(avg_profit_rate = min(avg_profit_rate))

### R-065:
商品データ（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。ただし、1円未満は切り捨てること。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）には欠損が生じていることに注意せよ。

In [ ]:
product %>%
mutate(target_price = trunc(unit_cost/0.7)) %>%
select(product_cd, unit_price, unit_cost, target_price) %>%
mutate(check_rate = (target_price - unit_cost)/target_price) %>%
head

### R-066:
商品データ（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。今回は、1円未満を丸めること（四捨五入または偶数への丸めで良い）。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）には欠損が生じていることに注意せよ。

In [ ]:
product %>%
mutate(target_price = round(unit_cost/0.7)) %>%
select(product_cd, unit_price, unit_cost, target_price) %>%
mutate(check_rate = (target_price - unit_cost)/target_price) %>%
head

### R-067:
商品データ（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。今回は、1円未満を切り上げること。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）には欠損が生じていることに注意せよ。

In [ ]:
product %>%
mutate(target_price = ceiling(unit_cost/0.7)) %>%
select(product_cd, unit_price, unit_cost, target_price) %>%
mutate(check_rate = (target_price - unit_cost)/target_price) %>%
head

### R-068:
商品データ（df_product）の各商品について、消費税率10％の税込み金額を求めよ。1円未満の端数は切り捨てとし、結果を10件表示せよ。ただし、単価（unit_price）には欠損が生じていることに注意せよ。

In [ ]:
product %>%
mutate(price_taxin = trunc(1.1 * unit_price)) %>%
select(unit_price, price_taxin) %>%
head

### R-069:
レシート明細データ（df_receipt）と商品データ（df_product）を結合し、顧客毎に全商品の売上金額合計と、カテゴリ大区分コード（category_major_cd）が"07"（瓶詰缶詰）の売上金額合計を計算の上、両者の比率を求めよ。抽出対象はカテゴリ大区分コード"07"（瓶詰缶詰）の売上実績がある顧客のみとし、結果を10件表示せよ。

In [ ]:
receipt_tbl %>%
inner_join(product_tbl, by = 'product_cd') %>% 
group_by(customer_id) %>%
summarise(
	amount_canned = sum(if_else(category_major_cd == '07',amount, 0), na.rm = TRUE),
	amount_total = sum(amount)
) %>%
filter(amount_canned > 0) %>%
mutate(canned_ratio = as.numeric(amount_canned) / amount_total) %>%
head(10)

### R-070:
レシート明細データ（df_receipt）の売上日（sales_ymd）に対し、顧客データ（df_customer）の会員申込日（application_date）からの経過日数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに10件表示せよ（sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。

In [ ]:
customer_tbl %>%
    inner_join(receipt_tbl %>% select(customer_id, sales_ymd), by = "customer_id") %>%
    select(customer_id, sales_ymd, application_date) %>%
    collect() %>%
    mutate(elapsed_days = lubridate::interval(lubridate::ymd(application_date), lubridate::ymd(sales_ymd)) %/% lubridate::days(1)) %>%
    head()

In [ ]:
customer_tbl %>%
    inner_join(receipt_tbl %>% select(customer_id, sales_ymd), by = "customer_id") %>%
    select(customer_id, sales_ymd, application_date) %>%
    collect() %>%
    mutate(interval = lubridate::interval(lubridate::ymd(application_date), lubridate::ymd(sales_ymd))) %>%
    head()

### R-071:
レシート明細データ（df_receipt）の売上日（sales_ymd）に対し、顧客データ（df_customer）の会員申込日（application_date）からの経過月数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに10件表示せよ（sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。1ヶ月未満は切り捨てること。

In [ ]:
# work in local
library(lubridate)

compare_date_local <- cul %>%
select(customer_id, application_date) %>%
inner_join(select(rel, customer_id, sales_ymd), by='customer_id') %>%
transmute(customer_id, sales_ymd = ymd(sales_ymd), application_date = ymd(application_date))

compare_date_local %>% 
mutate(elapsed_months = interval(application_date, sales_ymd) %/% months(1)) %>%
head

### R-072:
レシート明細データ（df_receipt）の売上日（df_customer）に対し、顧客データ（df_customer）の会員申込日（application_date）からの経過年数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに10件表示せよ（sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。1年未満は切り捨てること。

In [ ]:
customer_tbl %>%
inner_join(receipt_tbl %>% select(customer_id, sales_ymd), by='customer_id') %>%
select(customer_id, sales_ymd, application_date) %>%
collect() %>%
mutate(elapsed_years = lubridate::interval(lubridate::ymd(application_date), lubridate::ymd(sales_ymd)) %/% months(1)) %>%
head

In [ ]:
# work in local
compare_date_local %>%
mutate(elapsed_years = interval(application_date, sales_ymd) %/% years(1)) %>%
head

### R-073:
レシート明細データ（df_receipt）の売上日（sales_ymd）に対し、顧客データ（df_customer）の会員申込日（application_date）からのエポック秒による経過時間を計算し、顧客ID（customer_id）、売上日、会員申込日とともに10件表示せよ（なお、sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。なお、時間情報は保有していないため各日付は0時0分0秒を表すものとする。

In [ ]:
# work in local
compare_date_local %>%
mutate(elapsed_epoch = (sales_ymd - application_date) %>% as.numeric(units = 'secs')) %>%
head

In [ ]:
customer_tbl %>%
    inner_join(receipt_tbl %>% select(customer_id, sales_ymd), by = "customer_id") %>%
    select(customer_id, sales_ymd, application_date) %>%
    collect() %>%
    mutate(elapsed_months = lubridate::interval(lubridate::ymd(application_date), lubridate::ymd(sales_ymd)) %/% seconds(1)) %>%
    head()

### R-074:
レシート明細データ（df_receipt）の売上日（sales_ymd）に対し、当該週の月曜日からの経過日数を計算し、売上日、直前の月曜日付とともに10件表示せよ（sales_ymdは数値でデータを保持している点に注意）。

In [ ]:
receipt_tbl %>%
select(sales_ymd) %>%
collect() %>%
mutate(
	elapsed_weekdays = lubridate::ymd(sales_ymd) %>% lubridate::wday(week_start = 1) -1,
	last_monday = lubridate::ymd(sales_ymd) - elapsed_weekdays
	) %>%
	head

In [ ]:
library(lubridate)
re %>%
data.frame %>%
mutate(
	sales_ymd_d = sales_ymd %>% ymd,
	elapsed_weekdays = sales_ymd_d %>% wday(week_start = 1)-1,
	last_monday = sales_ymd_d - elapsed_weekdays
	) %>%
	head

### R-075:
顧客データ（df_customer）からランダムに1%のデータを抽出し、先頭から10件表示せよ。

In [ ]:
customer_tbl %>% slice_sample(n=customer_tbl %>% count() %>% collect() %>% as.numeric() %/% 100) %>% head

### R-076:
顧客データ（df_customer）から性別コード（gender_cd）の割合に基づきランダムに10%のデータを層化抽出し、性別コードごとに件数を集計せよ。

In [ ]:
sampled_customer <- customer_tbl %>% 
	group_by(gender_cd) %>%
	collect() %>%
	dplyr::slice_sample(prop = 0.1)
sampled_customer %>% nrow() %>% as.numeric()

### R-077:
レシート明細データ（df_receipt）の売上金額を顧客単位に合計し、合計した売上金額の外れ値を抽出せよ。なお、外れ値は売上金額合計を対数化したうえで平均と標準偏差を計算し、その平均から3σを超えて離れたものとする（自然対数と常用対数のどちらでも可）。結果は10件表示せよ。


In [ ]:
receipt_tbl %>%
group_by(customer_id) %>%
summarise(log_amount = log(sum(amount, na.rm = TRUE))) %>%
mutate(mean = mean(log_amount, na.rm = TRUE), sd = sd(log_amount,na.rm = TRUE)) %>%
filter(!(log_amount <= mean+3*sd)) %>%
head

### R-078:
レシート明細データ（df_receipt）の売上金額（amount）を顧客単位に合計し、合計した売上金額の外れ値を抽出せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。なお、ここでは外れ値を第1四分位と第3四分位の差であるIQRを用いて、「第1四分位数-1.5×IQR」を下回るもの、または「第3四分位数+1.5×IQR」を超えるものとする。結果は10件表示せよ。

In [ ]:
# assume you are on a local machine
library(stringr)

receipt_tbl %>%
filter(!str_detect(customer_id, '^Z')) %>%
group_by(customer_id) %>%
summarise(amount = sum(amount, na.rm = TRUE)) %>%
collect() %>%
filter(
	amount %>%
	{. > quantile(., probs = 0.25) + 1.5*IQR(.) | . < quantile(., probs = 0.25) - 1.5*IQR(.)}
) %>% head

In [ ]:
# assume you are on a local machine
library(stringr)

receipt_tbl %>%
filter(!str_detect(customer_id, '^Z')) %>%
group_by(customer_id) %>%
summarise(amount = sum(amount, na.rm = TRUE)) %>%
collect() %>%
filter(
	amount %>%
	{. > quantile(., probs = 0.25) + 1.5*IQR(.) | . < quantile(., probs = 0.25) - 1.5*IQR(.)}
) %>% head

In [ ]:
# assume you are on a local machine
rel <- DBI::dbReadTable(con, "receipt")
library(stringr)

rel %>%
filter(!str_detect(customer_id, '^Z')) %>%
group_by(customer_id) %>%
summarise(amount = sum(amount, na.rm = TRUE)) %>%
mutate(iqr = quantile(amount)[4] - quantile(amount)[2]) %>%
mutate(thre_lower = quantile(amount)[2] - 1.5*iqr, thre_upper = quantile(amount)[4] + 1.5*iqr) %>%
filter(amount < thre_lower | thre_upper < amount) %>%
head

### R-079:
商品データ（df_product）の各項目に対し、欠損数を確認せよ。

In [ ]:
product_tbl %>% filter(if_any(-c(), ~ is.na(.))) %>% collect() %>% 
summarise(across(everything(), ~ is.na(.) %>% sum()))

### R-080:
商品データ（df_product）のいずれかの項目に欠損が発生しているレコードを全て削除した新たな商品データを作成せよ。なお、削除前後の件数を表示させ、079で確認した件数だけ減少していることも確認すること。

In [ ]:
# missing data
product_tbl %>% filter(if_any(-c(), ~ is.na(.))) %>% head

In [ ]:
product_tbl %>% filter(if_all(-c(), ~ !is.na(.))) %>% head

In [ ]:
# complete.cases does not works on tbl
product_tbl %>% collect() %>%
filter(complete.cases(.)) %>% nrow()

### R-081:
単価（unit_price）と原価（unit_cost）の欠損値について、それぞれの平均値で補完した新たな商品データを作成せよ。なお、平均値については1円未満を丸めること（四捨五入または偶数への丸めで良い）。補完実施後、各項目について欠損が生じていないことも確認すること。

In [ ]:
library(tidymodels)

product_df <- data.frame(con, 'product')
product_tbl %>%
recipe() %>%
step_impute_mean(unit_price, unit_cost) %>%
prep() %>%
juice() %>%
head
# filter(unit_cost %>% is.na)

In [ ]:
product_tbl %>%
mutate(across(c(unit_price,unit_cost), ~ coalesce(mean(.x) %>% round())))

### R-082:
単価（unit_price）と原価（unit_cost）の欠損値について、それぞれの中央値で補完した新たな商品データを作成せよ。なお、中央値については1円未満を丸めること（四捨五入または偶数への丸めで良い）。補完実施後、各項目について欠損が生じていないことも確認すること。

In [ ]:
pr %>%
data.frame %>%
recipe() %>%
step_impute_median(unit_price, unit_cost) %>%
prep() %>%
juice() %>%
#head
filter(unit_cost %>% is.na) %>%head

### R-083:
単価（unit_price）と原価（unit_cost）の欠損値について、各商品のカテゴリ小区分コード（category_small_cd）ごとに算出した中央値で補完した新たな商品データを作成せよ。なお、中央値については1円未満を丸めること（四捨五入または偶数への丸めで良い）。補完実施後、各項目について欠損が生じていないことも確認すること。

In [17]:
product_tbl %>% 
collect() %>%
group_by(category_small_cd) %>%
group_modify(~ mutate(.x, 
	across(
		.cols = c(unit_price,unit_cost),
		.fns = ~ coalesce(.x, median(.x, na.rm = TRUE) %>% round())
))) %>% head

[1] 10030

In [2]:
product_tbl %>%
group_by(category_small_cd) %>%
summarise(
	across(
		.cols = c(unit_price, unit_cost),
		.fns = ~ median(.x) %>% round(),
		.names = '{.col}_med'
		) 
	) %>%
inner_join(product_tbl %>% filter(if_any(-c(), ~ is.na(.))), by = 'category_small_cd') %>%
collect() %>% 
group_by(category_small_cd) %>% 
group_modify(
	~ mutate(
		.x,
		across(
			.cols = c(unit_price,unit_cost),
			.fns = ~ coalesce(.x, cur_data() %>% summarise(across(stringr::str_c(cur_column(),'_med'),first)) %>% collect() %>% as.numeric()) 
			))
		) %>% 
select(- ends_with('_med')) %>% 
# temporarily save table in server.
copy_to(con, .,name = 'imputed', overwrite = TRUE)
product_tbl %>% filter(if_all(-c(), ~ !is.na(.))) %>% copy_to(con, .,name='product_complete', overwrite = TRUE)
# combine and show
rs <- DBI::dbSendQuery(con, "
create view product_imputed as
select *
from imputed
union all
select *
from product_complete
"
)
DBI::dbListTables(con)
tbl(con, 'product_imputed') %>% 
filter(if_all(-c(), ~ !is.na(.))) %>%
summarise(n())

Warning message:
“Missing values are always removed in SQL.
Use `median(x, na.rm = TRUE)` to silence this warning
This warning is displayed only once per session.”
NOTICE:  view "product_imputed" will be a temporary view


Warning message in result_create(conn@ptr, statement, immediate):
“Closing open result set, cancelling previous query”


[1] "customer"         "category"         "geocode"          "product"         
[5] "receipt"          "store"            "imputed"          "product_complete"
[9] "product_imputed"

# Source:   lazy query [?? x 1]
# Database: postgres [guest@localhost:5432/knock100]
    `n()`
  <int64>
1   10030


# Source:   lazy query [?? x 1]
# Database: postgres [guest@localhost:5432/knock100]
    `n()`
  <int64>
1       1

In [4]:
tbl(con, 'product_imputed') %>% 
filter(if_all(-c(), ~ !is.na(.))) %>%
summarise(n())

# Source:   lazy query [?? x 1]
# Database: postgres [guest@localhost:5432/knock100]
    `n()`
  <int64>
1   10030

In [ ]:
pr %>%
group_by(category_small_cd) %>%
summarise(
		price_median = unit_price %>% median(na.rm = TRUE) %>% round(),
		cost_median = unit_cost %>% median(na.rm = TRUE) %>% round()
		) %>%
inner_join(pr, by='category_small_cd') %>%
mutate(
	unit_price = ifelse(unit_price %>% is.na, price_median, unit_price),
	unit_cost = ifelse(unit_cost %>% is.na, cost_median, unit_cost)
)

### R-084:
顧客データ（df_customer）の全顧客に対して全期間の売上金額に占める2019年売上金額の割合を計算し、新たなデータを作成せよ。ただし、売上実績がない場合は0として扱うこと。そして計算した割合が0超のものを抽出し、結果を10件表示せよ。また、作成したデータに欠損が存在しないことを確認せよ。

In [24]:
receipt_tbl %>% 
mutate(year = sales_ymd %>% as.character() %>% substr(1,4)) %>%
group_by(customer_id) %>%
summarise(
	amount_all = sum(amount, na.rm=TRUE),
	amount_2019 = sum(if_else(year=='2019', coalesce(amount,0), 0), na.rm=TRUE)
)  %>%
 right_join(customer_tbl %>% select(customer_id), by = 'customer_id') %>%
 mutate(across(
 	.cols = starts_with('amount'),
 	.fns = ~ coalesce(.x, .0)
 )) %>% # the first of asked data
 filter(amount_2019>0) %>%
 # the second one
 mutate(ratio_2019 = amount_2019/amount_all) %>% head

# Source:   lazy query [?? x 4]
# Database: postgres [guest@localhost:5432/knock100]
  customer_id    amount_all amount_2019 ratio_2019
  <chr>               <dbl>       <dbl>      <dbl>
1 CS031415000172       5088        2971     0.584 
2 CS015414000103       3122         874     0.280 
3 CS011215000048       3444         248     0.0720
4 CS029415000023       5167        3767     0.729 
5 CS035415000029       7504        5823     0.776 
6 CS023513000066        771         208     0.270 

In [23]:
library(lubridate)
cu = tbl(con, 'customer')

re %>%
data.frame %>%
mutate(year = sales_ymd %>% ymd %>% year) %>%
group_by(customer_id, year) %>%
summarize(
	amount_all = sum(amount, na.rm = TRUE),
	amount_2019 = sum(ifelse(year == 2019,amount,0 ), na.rm = TRUE)
) %>%
right_join(cu %>% data.frame, by = 'customer_id') %>%
replace_na(list(amount_all = 0, amount_2019 = 0)) %>%
filter(amount_2019 > 0) %>% head(10)

ERROR: Error in replace_na(., list(amount_all = 0, amount_2019 = 0)): could not find function "replace_na"


### R-085:
顧客データ（df_customer）の全顧客に対し、郵便番号（postal_cd）を用いてジオコードデータ（df_geocode）を紐付け、新たな顧客データを作成せよ。ただし、1つの郵便番号（postal_cd）に複数の経度（longitude）、緯度（latitude）情報が紐づく場合は、経度（longitude）、緯度（latitude）の平均値を算出して使用すること。また、作成結果を確認するために結果を10件表示せよ。

In [33]:
geo_tbl %>% select(longitude, latitude, postal_cd)
group_by(postal_cd) %>%
summarise(across(
	.cols = c(longitude, latitude),
	.fns = ~ mean(.x, na.rm = TRUE)
)) %>%
right_join(customer_tbl %>% select(customer_id, postal_cd), by = 'postal_cd') %>%
head

# Source:   lazy query [?? x 3]
# Database: postgres [guest@localhost:5432/knock100]
   longitude latitude postal_cd
       <dbl>    <dbl> <chr>    
 1      141.     43.1 060-0000 
 2      141.     43.0 064-0941 
 3      141.     43.1 060-0042 
 4      141.     43.1 060-0042 
 5      141.     43.1 060-0042 
 6      141.     43.1 060-0042 
 7      141.     43.1 060-0042 
 8      141.     43.1 060-0042 
 9      141.     43.1 060-0042 
10      141.     43.1 060-0042 
# … with more rows

ERROR: Error in group_by(postal_cd): object 'postal_cd' not found


### R-086:
085で作成した緯度経度つき顧客データに対し、会員申込店舗コード（application_store_cd）をキーに店舗データ（df_store）と結合せよ。そして申込み店舗の緯度（latitude）・経度情報（longitude)と顧客住所（address）の緯度・経度を用いて申込み店舗と顧客住所の距離（単位：km）を求め、顧客ID（customer_id）、顧客住所（address）、店舗住所（address）とともに表示せよ。計算式は以下の簡易式で良いものとするが、その他精度の高い方式を利用したライブラリを利用してもかまわない。結果は10件表示せよ。

### R-087:
顧客データ（df_customer）では、異なる店舗での申込みなどにより同一顧客が複数登録されている。名前（customer_name）と郵便番号（postal_cd）が同じ顧客は同一顧客とみなして1顧客1レコードとなるように名寄せした名寄顧客データを作成し、顧客データの件数、名寄顧客データの件数、重複数を算出せよ。ただし、同一顧客に対しては売上金額合計が最も高いものを残し、売上金額合計が同一もしくは売上実績がない顧客については顧客ID（customer_id）の番号が小さいものを残すこととする。

### R-088:
087で作成したデータを元に、顧客データに統合名寄IDを付与したデータを作成せよ。ただし、統合名寄IDは以下の仕様で付与するものとする。
- 重複していない顧客：顧客ID（customer_id）を設定
- 重複している顧客：前設問で抽出したレコードの顧客IDを設定

顧客IDのユニーク件数と、統合名寄IDのユニーク件数の差も確認すること。

### R-089:
売上実績がある顧客を、予測モデル構築のため学習用データとテスト用データに分割したい。それぞれ8:2の割合でランダムにデータを分割せよ。

### R-090:
レシート明細データ（df_receipt）は2017年1月1日〜2019年10月31日までのデータを有している。売上金額（amount）を月次で集計し、学習用に12ヶ月、テスト用に6ヶ月の時系列モデル構築用データを3セット作成せよ。